In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
(X_train,Y_train),(X_test,Y_test) = keras.datasets.mnist.load_data()

In [3]:
print("X_train shape: ",X_train.shape)
print("Y_train shape ",Y_train.shape)
print("X_test shape: ",X_test.shape)
print("Y_test shape: ",Y_test.shape)

X_train shape:  (60000, 28, 28)
Y_train shape  (60000,)
X_test shape:  (10000, 28, 28)
Y_test shape:  (10000,)


In [4]:
X = np.append(X_train,X_test,axis=0)
Y = np.append(Y_train,Y_test,axis=0)

In [5]:
X = X.astype("float32")/255.0

In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state = 8)

In [7]:
X_train = X_train.reshape(X_train.shape[0],28,28,1)
X_test = X_test.reshape(X_test.shape[0],28,28,1)
X_train.shape, X_test.shape

((49000, 28, 28, 1), (21000, 28, 28, 1))

In [8]:
Y_train = Y_train.reshape(-1,)
Y_test = Y_test.reshape(-1,)

In [9]:
print("X_train shape: ",X_train.shape)
print("Y_train shape ",Y_train.shape)
print("X_test shape: ",X_test.shape)
print("Y_test shape: ",Y_test.shape)

X_train shape:  (49000, 28, 28, 1)
Y_train shape  (49000,)
X_test shape:  (21000, 28, 28, 1)
Y_test shape:  (21000,)


In [10]:
np.unique(Y_train,return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([4788, 5505, 4935, 5009, 4760, 4362, 4875, 5118, 4793, 4855]))

In [11]:
def get_pairs(X,Y):
  labels_index = []
  for i in range(10):
    idx = np.where(Y==i)[0]
    labels_index.append(idx)

  X_new = []
  Y_new = []
  for i in range(len(X)):
    simidx = np.random.choice(labels_index[Y[i]])
    diffidx = np.random.choice([x for x in range(len(labels_index)) if(x != i)])
    diffidx = np.random.choice(labels_index[diffidx])
    X_new.append([X[i],X[simidx]])
    Y_new.append(1)
    X_new.append([X[i],X[diffidx]])
    Y_new.append(0)
  
  return np.array(X_new),np.array(Y_new).astype("float32")



In [12]:
X_new,Y_new = get_pairs(X_train,Y_train)

In [13]:
def get_euc_dist(X):
  # print(X.shape)
  A,B = X
  # print(A.shape,B.shape)
  tempsum = K.sum(K.square(A-B),axis = 1,keepdims=True)
  return K.sqrt(K.maximum(tempsum,K.epsilon()))

In [14]:
Y_new.shape

(98000,)

In [15]:
def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

def compute_accuracy(y_true, y_pred):
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)

In [74]:
def get_siamese_model():
  imginput = keras.layers.Input(shape = (28,28,1))
  x = keras.layers.Conv2D(128, (3,3),padding="same", activation = 'relu')(imginput)
  x = keras.layers.MaxPool2D(pool_size = (3,3))(x)
  x = keras.layers.Dropout(0.1)(x)
  x = keras.layers.Conv2D(256, (3,3), padding="same",activation = 'relu')(x)
  x = keras.layers.MaxPool2D(pool_size = (3,3))(x)
  x = keras.layers.Dropout(0.1)(x)
  x = keras.layers.Flatten()(x)
  x = keras.layers.Dense(128, activation='relu')(x)
  x = keras.layers.Dropout(0.1)(x)
  x = keras.layers.Dense(128, activation='relu')(x)
  x = keras.layers.Dropout(0.1)(x)
  x = keras.layers.Dense(128, activation='relu')(x)
  x = keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
  x = keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
  dense = keras.models.Model(imginput, x)

  input1 = keras.layers.Input((28,28,1))
  input2 = keras.layers.Input((28,28,1))

  dense1 = dense(input1)
  dense2 = dense(input2)

  merge_layer = keras.layers.Lambda(get_euc_dist,output_shape=(lambda shapes: (shapes[0][0],1)))([dense1, dense2])
  # dense_layer = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

  return keras.models.Model(inputs=[input1, input2], outputs=merge_layer)

In [28]:
##LOSS FUNCTIONS:
##Constrastive Loss:

class ContrastiveLoss(keras.losses.Loss):
 
    def __init__(self, margin=1):
        super().__init__()
        self.margin = margin
 
    def call(self, y_true, y_pred):
        square_pred = K.square(y_pred)
        margin_square = K.square(K.maximum(self.margin - y_pred, 0))
        loss = K.mean(y_true * square_pred + (1 - y_true) * margin_square)
        return loss




In [75]:
contrastive_loss = ContrastiveLoss(margin=1)
siamese = get_siamese_model()
siamese.compile(loss = contrastive_loss ,optimizer="adam", metrics=[accuracy])

In [32]:
siamese.fit([X_new[:,0],X_new[:,1]],Y_new,batch_size=128,epochs= 10,shuffle=True)

Epoch 1/10
766/766 [==============================] - 18s 22ms/step - loss: 0.0589 - accuracy: 0.9335
Epoch 2/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0539 - accuracy: 0.9392
Epoch 3/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0516 - accuracy: 0.9421
Epoch 4/10
766/766 [==============================] - 16s 21ms/step - loss: 0.0499 - accuracy: 0.9443
Epoch 5/10
766/766 [==============================] - 16s 21ms/step - loss: 0.0499 - accuracy: 0.9445
Epoch 6/10
766/766 [==============================] - 16s 22ms/step - loss: 0.0503 - accuracy: 0.9437
Epoch 7/10
766/766 [==============================] - 16s 21ms/step - loss: 0.0493 - accuracy: 0.9453
Epoch 8/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0481 - accuracy: 0.9466
Epoch 9/10
766/766 [==============================] - 16s 22ms/step - loss: 0.0478 - accuracy: 0.9472
Epoch 10/10
766/766 [==============================] - 16s 21ms/step - loss: 0.049

In [33]:

X_new_test,Y_new_test = get_pairs(X_test,Y_test)

In [34]:
siamese.evaluate([X_new_test[:,0],X_new_test[:,1]],Y_new_test[:])

1313/1313 [==============================] - 5s 4ms/step - loss: 0.0546 - accuracy: 0.9407


[0.05456561595201492, 0.9407130479812622]

In [37]:
Y_pred = siamese.predict([X_new_test[:, 0], X_new_test[:, 1]])
compute_accuracy(Y_new_test, Y_pred)

0.9407142857142857

In [58]:
### Triplet Loss

class TripletLoss(keras.layers.Layer):
 
    def __init__(self, margin=0.2):
        super().__init__()
        self.margin = margin
 
    def call(self, x):
      anchor,positive,negative = x
      pos_dist = K.sum(K.square(anchor-positive),axis=1)
      neg_dist = K.sum(K.square(anchor-negative),axis=1)
      basic_loss = pos_dist-neg_dist+self.margin
      loss = K.maximum(basic_loss,0.0)
      return loss

In [69]:

def get_triplets(X,Y,labels_index):
  simidx = np.random.choice(len(labels_index))
  diffidx = np.random.choice([x for x in range(len(labels_index)) if(x != simidx)])
  n = np.random.choice(labels_index[simidx])
  a = np.random.choice(labels_index[diffidx])
  p = np.random.choice(labels_index[diffidx])
  return n,a,p

def generate_triplets(X,Y,batch_size,labels_index):
    while True:
        list_a = []
        list_p = []
        list_n = []

        for i in range(batch_size):
            a, p, n = get_triplets(X,Y,labels_index)
            list_a.append(X[a])
            list_p.append(X[p])
            list_n.append(X[n])
            
        A = np.array(list_a).astype("float32")
        P = np.array(list_p).astype("float32")
        N = np.array(list_n).astype("float32")
        label = np.ones(batch_size)
        yield [A, P, N], label

In [60]:
def identity_loss(y_true, y_pred):
    return K.mean(y_pred)

In [61]:
imginput = keras.layers.Input(shape = (28,28,1))
x = keras.layers.Conv2D(128, (3,3),padding="same", activation = 'relu')(imginput)
x = keras.layers.MaxPool2D(pool_size = (3,3))(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Conv2D(256, (3,3), padding="same",activation = 'relu')(x)
x = keras.layers.MaxPool2D(pool_size = (3,3))(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
x = keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
dense = keras.models.Model(imginput, x)

input1 = keras.layers.Input((28,28,1))
input2 = keras.layers.Input((28,28,1))
input3 = keras.layers.Input((28,28,1))

dense1 = dense(input1)
dense2 = dense(input2)
dense3 = dense(input3)

merge_layer = TripletLoss()([dense1, dense2, dense3]) 
# dense_layer = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

siamese_triplet = keras.models.Model(inputs=[input1, input2,input3], outputs=merge_layer)

In [62]:
siamese_triplet.compile(loss=identity_loss)
siamese_triplet.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
model_10 (Functional)           (None, 128)          624512      input_20[0][0]                   
                                                                 input_21[0][0]            

In [72]:
labels_index = []
for i in range(10):
  idx = np.where(Y_train==i)[0]
  labels_index.append(idx)
train_generator = generate_triplets(X_train, Y_train,batch_size=128,labels_index = labels_index)

labels_index = []
for i in range(10):
  idx = np.where(Y_test==i)[0]
  labels_index.append(idx)
test_generator = generate_triplets(X_test, Y_test,batch_size=128,labels_index = labels_index)

In [73]:
siamese_triplet.fit(train_generator,
                    validation_data=test_generator, 
                    epochs=10, 
                    verbose=2,steps_per_epoch=20, 
                    validation_steps=30)

Epoch 1/10
20/20 - 2s - loss: 0.2026 - val_loss: 0.2000
Epoch 2/10
20/20 - 1s - loss: 0.2014 - val_loss: 0.2001
Epoch 3/10
20/20 - 1s - loss: 0.1988 - val_loss: 0.2000
Epoch 4/10
20/20 - 1s - loss: 0.2008 - val_loss: 0.1999
Epoch 5/10
20/20 - 1s - loss: 0.2003 - val_loss: 0.2001
Epoch 6/10
20/20 - 1s - loss: 0.2020 - val_loss: 0.2000
Epoch 7/10
20/20 - 1s - loss: 0.2020 - val_loss: 0.2000
Epoch 8/10
20/20 - 1s - loss: 0.2000 - val_loss: 0.2000
Epoch 9/10
20/20 - 1s - loss: 0.2006 - val_loss: 0.2000
Epoch 10/10
20/20 - 1s - loss: 0.2002 - val_loss: 0.2000


In [81]:
###Adam Optimizer

contrastive_loss = ContrastiveLoss(margin=1)
adam = get_siamese_model()
adam.compile(loss = contrastive_loss ,optimizer="adam", metrics=[accuracy])

In [82]:
adam.fit([X_new[:, 0], X_new[:, 1]], Y_new, batch_size=128, epochs=10)

Epoch 1/10
766/766 [==============================] - 18s 22ms/step - loss: 0.1596 - accuracy: 0.7491
Epoch 2/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0583 - accuracy: 0.9344
Epoch 3/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0536 - accuracy: 0.9396
Epoch 4/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0507 - accuracy: 0.9434
Epoch 5/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0500 - accuracy: 0.9445
Epoch 6/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0491 - accuracy: 0.9453
Epoch 7/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0483 - accuracy: 0.9465
Epoch 8/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0477 - accuracy: 0.9471
Epoch 9/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0486 - accuracy: 0.9459
Epoch 10/10
766/766 [==============================] - 17s 22ms/step - loss: 0.048

In [84]:
Y_pred = adam.predict([X_new_test[:, 0], X_new_test[:, 1]])
compute_accuracy(Y_new_test, Y_pred)

0.9381904761904762

In [85]:
###RMSProp Optimizer

contrastive_loss = ContrastiveLoss(margin=1)
rms = get_siamese_model()
rms.compile(loss = contrastive_loss ,optimizer="rmsprop", metrics=[accuracy])

In [86]:
rms.fit([X_new[:, 0], X_new[:, 1]], Y_new, batch_size=128, epochs=10)

Epoch 1/10
766/766 [==============================] - 18s 22ms/step - loss: 0.1338 - accuracy: 0.8084
Epoch 2/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0565 - accuracy: 0.9355
Epoch 3/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0527 - accuracy: 0.9403
Epoch 4/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0515 - accuracy: 0.9425
Epoch 5/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0502 - accuracy: 0.9437
Epoch 6/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0496 - accuracy: 0.9446
Epoch 7/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0503 - accuracy: 0.9441
Epoch 8/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0492 - accuracy: 0.9452
Epoch 9/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0485 - accuracy: 0.9462
Epoch 10/10
766/766 [==============================] - 17s 22ms/step - loss: 0.048

In [87]:
Y_pred = rms.predict([X_new_test[:, 0], X_new_test[:, 1]])
compute_accuracy(Y_new_test, Y_pred)

0.9393333333333334

In [89]:
###Hyperparameters Optimization:

lr_scheduler = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-3, 
                                             decay_steps=10000, 
                                             decay_rate=0.9)

In [91]:
rmsopm = keras.optimizers.RMSprop(learning_rate=lr_scheduler)
contrastive_loss = ContrastiveLoss(margin=1)
rms = get_siamese_model()
rms.compile(loss = contrastive_loss ,optimizer="rmsprop", metrics=[accuracy])

In [92]:
rms.fit([X_new[:, 0], X_new[:, 1]], Y_new, batch_size=128, epochs=10)

Epoch 1/10
766/766 [==============================] - 18s 22ms/step - loss: 0.1310 - accuracy: 0.8129
Epoch 2/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0562 - accuracy: 0.9358
Epoch 3/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0533 - accuracy: 0.9396
Epoch 4/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0517 - accuracy: 0.9418
Epoch 5/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0501 - accuracy: 0.9437
Epoch 6/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0503 - accuracy: 0.9440
Epoch 7/10
766/766 [==============================] - 17s 23ms/step - loss: 0.0499 - accuracy: 0.9445
Epoch 8/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0495 - accuracy: 0.9445
Epoch 9/10
766/766 [==============================] - 17s 22ms/step - loss: 0.0489 - accuracy: 0.9457
Epoch 10/10
766/766 [==============================] - 17s 22ms/step - loss: 0.048

In [93]:
Y_pred = rms.predict([X_new_test[:, 0], X_new_test[:, 1]])
compute_accuracy(Y_new_test, Y_pred)

0.9375476190476191

In [ ]:
# Pros and Cons of the Siamese Network


**Pros:**

*  **More Robust to class Imbalance:** With the aid of One-shot learning, given a few images per class is sufficient for Siamese Networks to recognize those images in the future
*   **Learning from Semantic Similarity:** Siamese focuses on learning embeddings (in the deeper layer) that place the same classes/concepts close together. Hence, can learn semantic similarity.
*   **Nice to an ensemble with the best classifier:** Given that its learning mechanism is somewhat different from Classification, simple averaging of it with a Classifier can do much better than average 2 correlated Supervised models (e.g. GBM & RF classifier)



**Cons:**


*   **Needs more training time than normal networks**: Since Siamese Networks involves quadratic pairs to learn from (to see all information available) it is slower than normal classification type of learning(pointwise learning)
*   **Doesn’t output probabilities**: Since training involves pairwise learning, it won’t output the probabilities of the prediction, but the distance from each class






